# IMPORT PANDA & DATA RETRIVAL

In [2]:
import pandas as pd
# turn of warning messages
pd.options.mode.chained_assignment = None  # default='warn'

# get data
df = pd.read_csv('student_records.csv')
df

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


# Feature extraction and engineering

In [3]:
# get features 
feature_names = ['OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']
training_features = df[feature_names]
#Assigning Outcome name and variable
outcome_name = ['Recommend']
outcome_labels = df[outcome_name]

View Features

In [4]:
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


View outcome

In [5]:
outcome_labels

,Recommend
0,Yes
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No


# Categorising features as per data tye

In [6]:
numeric_feature_names = ['ResearchScore', 'ProjectScore']
categoricial_feature_names = ['OverallGrade', 'Obedient']

# Numeric Feature Scaling

IMPORTING LIBRARY FUNCTION FOR SCALING

In [8]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

Fiting Scalar 

In [9]:
ss.fit(training_features[numeric_feature_names])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

Scaling Features

In [10]:
training_features[numeric_feature_names] = ss.transform(training_features[numeric_feature_names])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


View data for scaled features

In [11]:
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


# Engineering Categorical Features

Assign dummy variable for categorical data

In [12]:
training_features = pd.get_dummies(training_features, columns=categoricial_feature_names)

view the all engineered features

In [13]:
training_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,1,0,0,0,0,0,1
1,0.730648,-0.091777,0,0,1,0,0,1,0
2,-1.803390,-1.560203,0,0,0,0,1,1,0
3,0.392776,0.772004,0,1,0,0,0,0,1
4,-1.465519,-0.998746,0,0,0,1,0,1,0
5,0.967158,1.117516,1,0,0,0,0,0,1
6,-0.114032,0.253735,0,1,0,0,0,0,1
7,0.392776,-0.869179,0,0,1,0,0,0,1


In [14]:
categorical_engineered_features = list(set(training_features.columns) - set(numeric_feature_names))

# Data Modeling

Importing logistic regression and numpy libraries

In [16]:
from sklearn.linear_model import LogisticRegression
import numpy as np

Model fitting

In [17]:
lr = LogisticRegression() 
model = lr.fit(training_features, np.array(outcome_labels['Recommend']))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Model afer regression

In [18]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

# Model Evaluation

Accuracy

In [19]:
pred_labels = model.predict(training_features)
actual_labels = np.array(outcome_labels['Recommend'])

In [21]:
from sklearn.metrics import accuracy_score
print('Accuracy:', float(accuracy_score(actual_labels, pred_labels))*100, '%')

Accuracy: 100.0 %


Classification

In [22]:
from sklearn.metrics import classification_report
print('Classification Stats:')
print(classification_report(actual_labels, pred_labels))

Classification Stats:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

   micro avg       1.00      1.00      1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



# Model Deployment

Import library function for deployment

In [23]:
from sklearn.externals import joblib
import os

Saving Models on Server

In [28]:
if not os.path.exists('Model'):
    os.mkdir('Model')
if not os.path.exists('Scaler'):
    os.mkdir('Scaler') 
    
joblib.dump(model, r'Model/model.pickle') 
joblib.dump(ss, r'Scaler/scaler.pickle')

['Scaler/scaler.pickle']

# Prediction

Model loading

In [29]:
model = joblib.load(r'Model/model.pickle')
scaler = joblib.load(r'Scaler/scaler.pickle')

Data Retrival

In [30]:
new_data = pd.DataFrame([{'Name': 'Nathan', 'OverallGrade': 'F', 'Obedient': 'N', 'ResearchScore': 30, 'ProjectScore': 20},
                  {'Name': 'Thomas', 'OverallGrade': 'A', 'Obedient': 'Y', 'ResearchScore': 78, 'ProjectScore': 80}])
new_data = new_data[['Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']]
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore
0,Nathan,F,N,30,20
1,Thomas,A,Y,78,80


Preparing data for prediction

In [32]:
prediction_features = new_data[feature_names]

Scaling

In [33]:
prediction_features[numeric_feature_names] = scaler.transform(prediction_features[numeric_feature_names])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


Categorical variable engineering

In [34]:
prediction_features = pd.get_dummies(prediction_features, columns=categoricial_feature_names)

View predictions

In [35]:
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y
0,-1.127647,-1.430636,0,1,1,0
1,0.494137,1.160705,1,0,0,1


Add missing categorical features

In [36]:
current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features
for feature in missing_features:
    prediction_features[feature] = [0] * len(prediction_features) 

Final Prediction

In [37]:
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y,OverallGrade_E,OverallGrade_B,OverallGrade_C
0,-1.127647,-1.430636,0,1,1,0,0,0,0
1,0.494137,1.160705,1,0,0,1,0,0,0


Prediction using model

In [38]:
predictions = model.predict(prediction_features)
new_data['Recommend'] = predictions
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Nathan,F,N,30,20,No
1,Thomas,A,Y,78,80,Yes
